# Generar red promedio para trayectorias

In [61]:
# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("/home/serch/pdbmani/Serch/math_tricks/")
sys.path.append("/home/serch/pdbmani/Serch/")
import math_vect_tools as mymath
# herramientas para leer pdbs
import read_pdb_tools as rpt
# funciones de click generadas en pandas
import funciones_CLICK as fc
# cuenta tiempo de ejecucion
# networks
import networkx as nx

import os
os.chdir('/home/serch/pdbmani/Serch')

In [62]:
# lectura de archivo
file1 = '/home/serch/pdbmani/Serch/pdbs/dimer_con_cofact_clean.pdb'  # dime_out no es trayectoria es pdb

# se define la estructura
trj1 = rpt.Trajectory(file1)

# se lee el pdb y se agrega al objeto
trj1.ReadTraj("%s" % file1)

trj11 = trj1.frames

# set center_mass calculando aqui tarda un poco
for pdbstruct in trj11:
    pdbstruct.set_center_mass()

In [103]:
def get_df_distancias(ref):
    """Funcion para obtener los enlaces de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo
#     enlaces = [ [res1.resi, res2.resi] for res1 in ref[1:-1] for res2 in ref[1:-1] 
#                if res2.resi >= res1.resi if mymath.distance(res2.GetAtom('CA').coord, res1.GetAtom('CA').coord) < 10
#               ]

    enlaces = []
    for res1 in ref:
        for res2 in ref:
            if res1.resi in [1,178, 201,378] or res2.resi in [1, 178, 201,378]: # se hizo a  mano el quitar los atomos que no nos interesn
                continue
            else:
                if res2.resi >= res1.resi:
                    dist = mymath.distance(res2.GetAtom('CA').coord, res1.GetAtom('CA').coord)
                    if  dist < 10:
                        enlaces.append([res1.resi, res2.resi])

    # se genera la matriz de adyacencias para la red
    return enlaces

In [104]:
pdbs = [pdb.GetResChain() for pdb in trj11]

enlaces = [get_df_distancias(residues) for residues in pdbs]
# aqui se filtran los enlaces

%time

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [105]:
from collections import Counter # obtener conteo de enlaces
a = Counter([str(j) for i in enlaces for j in i]).most_common()

import ast # para pasar de str a tupla o listas
red_promedio = nx.Graph([ast.literal_eval(i[0]) for i in a if (i[1] / len(enlaces)) > 0.7])

print(nx.info(red_promedio))

trj11[0].PrintPdbInfo()

Name: 
Type: Graph
Number of nodes: 352
Number of edges: 3195
Average degree:  18.1534
Number of residues and frame: 356    1
Number of chains:             1  dict_keys(['A'])


# Genera red y subred de vecinos secundarios para Dimero de interes

In [65]:
%time
#!/usr/bin/env python

# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("/home/serch/pdbmani/Serch/math_tricks/")
sys.path.append("/home/serch/pdbmani/Serch/")
import math_vect_tools as mymath
# herramientas para leer pdbs
import read_pdb_tools as rpt
# funciones de click generadas en pandas
import funciones_CLICK as fc
# cuenta tiempo de ejecucion
import datetime
time_all = datetime.datetime.now()
# networks
import networkx as nx
import os
os.chdir('/home/serch/pdbmani/Serch')


# lectura de archivo
file1 = '/home/serch/pdbmani/Serch/pdbs/dime_out.pdb'  # dime_out no es trayectoria es pdb

# se define la estructura
pdb1 = rpt.PdbStruct(file1)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)

pdb1.set_center_mass()

pdb1.Set_SS()

pdb1.GetNeighbors()

pdb11 = pdb1.GetResChain()

def get_df_distancias(ref):
    """Funcion para obtener los enlaces de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo

    # calcula distancia y regresa dataframe
    enlaces = []
    for res1 in ref[1:-1]:
        for res2 in ref[1:-1]:
            if res2.resi >= res1.resi:
                dist = mymath.distance(res2.GetAtom('CA').coord, res1.GetAtom('CA').coord)
                if dist < 10 and dist != 0:
                    enlaces.append([res1.resi, res2.resi])

    # se genera la matriz de adyacencias para la red
    return enlaces

enlaces = get_df_distancias(pdb11)
# red de todo el dimero.
red = nx.Graph(enlaces)
nx.set_edge_attributes(red, 'red', name='color')

# nx.write_gexf(red, 'r.gexf')

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [20]:
# nx.write_gexf(red, 'r.gexf')

In [66]:
print(nx.info(red))

Name: 
Type: Graph
Number of nodes: 174
Number of edges: 1500
Average degree:  17.2414


In [34]:
# funcion subred segundos vecinos
def get_second_nei(res, flag_write=False):
    first_nei = pdb1.GetRes(res).ngb
    list_second_nei = [pdb1.GetRes(residue).ngb for residue in first_nei]
    set_second_nei = set([j for i in list_second_nei for j in i])
    sub_red = red.subgraph(set_second_nei)
    nx.set_edge_attributes(subred, 'blue', name='color')
    if flag_write:
        nx.write_gexf(sub_red, 'r1.gexf')
        return sub_red
    else:
        return sub_red

In [58]:
en1 = [i for i in get_second_nei(160).edges]
en2 = [i for i in get_second_nei(5).edges]

In [59]:
len(en1)

571

In [60]:
len(en2)

296

In [63]:
# que enlaces si estan en uno u otro residuo
len([i for i in en1 if i in en2])

264

In [64]:
# que enlaces si estan en uno u otro residuo
len([i for i in en1 if i not in en2])

307

# CHeck de red con articulo Perica

In [110]:
%time
#!/usr/bin/env python

# librerias que utilizaras
import numpy as np
# por si no te lee las tools o functions creadas
import sys
sys.path.append("/home/serch/pdbmani/Serch/math_tricks/")
sys.path.append("/home/serch/pdbmani/Serch/")
import math_vect_tools as mymath
# herramientas para leer pdbs
import read_pdb_tools as rpt
# funciones de click generadas en pandas
import funciones_CLICK as fc
# cuenta tiempo de ejecucion
import datetime
time_all = datetime.datetime.now()
# networks
import networkx as nx
import os
os.chdir('/home/serch/pdbmani/Serch')


# lectura de archivo
file1 = '/home/serch/pdbmani/Serch/networks_scripts/pdbs/4p81.pdb'  # dime_out no es trayectoria es pdb

# se define la estructura
pdb1 = rpt.PdbStruct(file1)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)

pdb1.set_center_mass()

pdb1.Set_SS()

pdb1.GetNeighbors()

pdb11 = pdb1.GetResChain()

def get_df_distancias(ref):
    """Funcion para obtener los enlaces de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo

    # calcula distancia y regresa dataframe
    enlaces = []
    for res1 in ref[1:-1]:
        for res2 in ref[1:-1]:
            if res2.resi >= res1.resi:
                dist = mymath.distance(res2.GetAtom('CA').coord, res1.GetAtom('CA').coord)
                if dist < 10 and dist != 0:
                    enlaces.append([res1.resi, res2.resi])

    # se genera la matriz de adyacencias para la red
    return enlaces

enlaces = get_df_distancias(pdb11)
# red de todo el dimero.
red = nx.Graph(enlaces)
nx.set_edge_attributes(red, 'red', name='color')

nx.write_gexf(red, 'r.gexf')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [98]:
# nx.write_gexf(red, 'r.gexf')

In [99]:
print(nx.info(red))

Name: 
Type: Graph
Number of nodes: 164
Number of edges: 1439
Average degree:  17.5488


In [104]:
# funcion subred segundos vecinos
def get_second_nei(res, flag_write=False):
    first_nei = pdb1.GetRes(res).ngb
    list_second_nei = [pdb1.GetRes(residue).ngb for residue in first_nei]
    set_second_nei = set([j for i in list_second_nei for j in i])
    sub_red = red.subgraph(set_second_nei)
    nx.set_edge_attributes(sub_red, 'blue', name='color')
    
    if flag_write:
        nx.write_gexf(sub_red, 'r1.gexf')
        return sub_red
    else:
        return sub_red

In [105]:
en1 = [i for i in get_second_nei(160).edges]
en2 = [i for i in get_second_nei(5).edges]

In [106]:
len(en1)

250

In [107]:
len(en2)

222

In [108]:
# que enlaces si estan en uno u otro residuo
len([i for i in en1 if i in en2])

199

In [109]:
# que enlaces si estan en uno u otro residuo
len([i for i in en1 if i not in en2])

51

# Generando red de articulo Universality in Protein Residue Networks 1ash

In [88]:
# lectura de archivo
file1 = '/home/serch/pdbmani/Serch/networks_scripts/pdbs/1ash.pdb'  # dime_out no es trayectoria es pdb

# se define la estructura
pdb1 = rpt.PdbStruct(file1)

# se lee el pdb y se agrega al objeto
pdb1.AddPdbData("%s" % file1)

pdb1.set_center_mass()

pdb1.Set_SS()

pdb1.GetNeighbors()

pdb11 = pdb1.GetResChain()

def get_df_distancias(ref):
    """Funcion para obtener los enlaces de distancias de cada residuo
    Dudas en codigo pueden revisar fc.distancia_entre_atomos en ese se basa
    esta funcion, la diferencia es que se crea con el objeto residuo"""
    # se generan listas con coordenadas y numero de atomo

    # calcula distancia y regresa dataframe
    enlaces = []
    for res1 in ref[1:-1]:
        for res2 in ref[1:-1]:
            if res2.resi >= res1.resi:
                dist = mymath.distance(res2.GetAtom('CA').coord, res1.GetAtom('CA').coord)
                if dist < 7 and dist != 0:
                    enlaces.append([res1.resi, res2.resi])

    # se genera la matriz de adyacencias para la red
    return enlaces

enlaces = get_df_distancias(pdb11)
# red de todo el dimero.
red = nx.Graph(enlaces)
# nx.set_edge_attributes(red, 'red', name='color')

nx.write_gexf(red, 'r.gexf')

In [89]:
print(nx.info(red))

Name: 
Type: Graph
Number of nodes: 145
Number of edges: 569
Average degree:   7.8483
